In [29]:
failed_urls = []

In [30]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import os
import re
# Set working directory (adjust to your local path if needed)
os.chdir(r"C:\Users\acer\Downloads")
# Base URL for pagination
base_url = "https://numeum.fr/en/annuaire-des-adherents?page="

# Placeholder list to store the scraped names
adherent_names = []
urls = []
# Loop through pages from 0 to 170 (adjusted here to page 0 for testing)
for page in range(0,175):
    url = f"{base_url}{page}"
    response = requests.get(url)
    
    # Check if request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
            
        # Find all adherence items on the page
        adherents = soup.find_all("a", class_="subscriber")
        
        for adherent in adherents:
            # Locate the span containing the desired name (e.g., #NOCOMMENT)
            name_tag = adherent.find("span", class_="field field--name-title field--type-string field--label-hidden")
            
            # Check if the tag was found and extract the text
            if name_tag:
                name = name_tag.text.strip()
                adherent_names.append(name)
                name = re.sub(r'[^A-Za-z0-9\- ]', '', name)
                urls.append('https://numeum.fr/societe/'+name.lower().replace(' ','-'))
    
    # Optional: wait a bit between requests to avoid overwhelming the server
    # time.sleep(1)

# Save the data to a CSV file
df = pd.DataFrame(adherent_names, columns=["Adherent Name"])
print(df.head())
print(urls[:10])
df['URL'] = urls
df.to_csv("adherent_names.csv", index=False)
print("Scraping completed and data saved to adherent_names.csv")
df.head(30)


        Adherent Name
0          #NOCOMMENT
1          1DAY1LEARN
2  1G-LINK CONSULTING
3     1SPATIAL FRANCE
4  20-20 TECHNOLOGIES
['https://numeum.fr/societe/nocomment', 'https://numeum.fr/societe/1day1learn', 'https://numeum.fr/societe/1g-link-consulting', 'https://numeum.fr/societe/1spatial-france', 'https://numeum.fr/societe/20-20-technologies', 'https://numeum.fr/societe/22h22advisor', 'https://numeum.fr/societe/2si-consulting', 'https://numeum.fr/societe/360smartconnect', 'https://numeum.fr/societe/38-labs', 'https://numeum.fr/societe/3c-experts']
Scraping completed and data saved to adherent_names.csv


,Adherent Name,URL
0,#NOCOMMENT,https://numeum.fr/societe/nocomment
1,1DAY1LEARN,https://numeum.fr/societe/1day1learn
2,1G-LINK CONSULTING,https://numeum.fr/societe/1g-link-consulting
3,1SPATIAL FRANCE,https://numeum.fr/societe/1spatial-france
4,20-20 TECHNOLOGIES,https://numeum.fr/societe/20-20-technologies
5,22H22ADVISOR,https://numeum.fr/societe/22h22advisor
6,2SI CONSULTING,https://numeum.fr/societe/2si-consulting
7,360SMARTCONNECT,https://numeum.fr/societe/360smartconnect
8,38 LABS,https://numeum.fr/societe/38-labs
9,3C EXPERTS,https://numeum.fr/societe/3c-experts


In [31]:
print(len(df))

2052


In [32]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

# Base URL and headers
headers = {"User-Agent": "Mozilla/5.0"}

# Lists to store extracted data
emails = []
names = []
urls = []
failed_urls = []
failed_names = []

# Loop through each row in df to access company names and URLs
for index, row in df.iterrows():
    company_name = row["Adherent Name"]
    company_url = row["URL"]
    email_found = False  # Flag to check if email is found

    try:
        # Make a GET request to fetch the company's page
        response = requests.get(company_url, headers=headers)
        
        # Continue to the next iteration if request fails
        if response.status_code != 200:
            print(f"Failed to retrieve data for {company_name}")
            failed_urls.append(company_url)
            failed_names.append(company_name)
            continue

        # Parse the page content
        soup = BeautifulSoup(response.text, "html.parser")

        # Find all contact buttons with mailto: links
        contact_buttons = soup.find_all("a", href=re.compile(r"^mailto:"))

        # Extract email addresses from mailto: links
        for button in contact_buttons:
            email = button.get("href").replace("mailto:", "")
            # Append email, name, and URL if email is new
            if email not in emails:
                emails.append(email)
                names.append(company_name)
                urls.append(company_url)
                email_found = True  # Email was found for this company

        # If no email was found, record the company with an empty email
        if not email_found:
            emails.append("")  # Placeholder for no email
            names.append(company_name)
            urls.append(company_url)

    except Exception as e:
        print(f"An error occurred for {company_name}: {e}")
        failed_urls.append(company_url)
        failed_names.append(company_name)

# Create a DataFrame to store the results
result_df = pd.DataFrame({
    "Adherent Name": names,
    "Email": emails,
    "URL": urls
})

# Save the results to a CSV file
result_df.to_csv("adherent_names_complete.csv", index=False)

# Display the first 50 rows
result_df.head(50)


Failed to retrieve data for 573° CONSULTING
Failed to retrieve data for 9ALTITUDES
Failed to retrieve data for A-SYSTEMS
Failed to retrieve data for ABVSM - COGNILEARNING
Failed to retrieve data for ACATUS INFORMATIQUE
Failed to retrieve data for ACII
Failed to retrieve data for ADDACTIS FRANCE
Failed to retrieve data for AF2M
Failed to retrieve data for AGILITECH
Failed to retrieve data for AKKODIS
Failed to retrieve data for ALB CONSEIL
Failed to retrieve data for ALDEBARAN
Failed to retrieve data for ALLÉA SOLUTIONS
Failed to retrieve data for AMAZON WEB SERVICES (AWS)
Failed to retrieve data for ANAPLAN FRANCE
Failed to retrieve data for ANTHEA CONSEIL
Failed to retrieve data for AOTA
Failed to retrieve data for APODIS
Failed to retrieve data for APOLLON & CIE
Failed to retrieve data for AQUILOG
Failed to retrieve data for AS-TECH SOLUTIONS
Failed to retrieve data for ASAP TECHNOLOGIES GROUPE
Failed to retrieve data for ASE2I - LORENZ AND HAMILTON GROUP
Failed to retrieve data for 

,Adherent Name,Email,URL
0,#NOCOMMENT,laura.salin@nocomment.social,https://numeum.fr/societe/nocomment
1,1DAY1LEARN,eric.gabas@1day1learn.com,https://numeum.fr/societe/1day1learn
2,1G-LINK CONSULTING,contact@1glink.fr,https://numeum.fr/societe/1g-link-consulting
3,1SPATIAL FRANCE,yael.poncet@1spatial.com,https://numeum.fr/societe/1spatial-france
4,20-20 TECHNOLOGIES,france@2020spaces.com,https://numeum.fr/societe/20-20-technologies
5,22H22ADVISOR,remy@22h22advisor.com,https://numeum.fr/societe/22h22advisor
6,2SI CONSULTING,contact@2siconsulting.com,https://numeum.fr/societe/2si-consulting
7,360SMARTCONNECT,rm@360sc.io,https://numeum.fr/societe/360smartconnect
8,38 LABS,yasir@38labs.com,https://numeum.fr/societe/38-labs
9,3C EXPERTS,contact@3cexperts.fr,https://numeum.fr/societe/3c-experts


In [33]:
result_df.head()

,Adherent Name,Email,URL
0,#NOCOMMENT,laura.salin@nocomment.social,https://numeum.fr/societe/nocomment
1,1DAY1LEARN,eric.gabas@1day1learn.com,https://numeum.fr/societe/1day1learn
2,1G-LINK CONSULTING,contact@1glink.fr,https://numeum.fr/societe/1g-link-consulting
3,1SPATIAL FRANCE,yael.poncet@1spatial.com,https://numeum.fr/societe/1spatial-france
4,20-20 TECHNOLOGIES,france@2020spaces.com,https://numeum.fr/societe/20-20-technologies


In [34]:
# import requests
# from bs4 import BeautifulSoup
# import re
# import pandas as pd

# # Base URL and headers
# headers = {"User-Agent": "Mozilla/5.0"}

# # Lists to store extracted data
# emails = []
# names = []
# urls = []
# failed_urls = []
# failed_names = []
adf

# # Loop through each row in df to access company names and URLs
# for index, row in df.iterrows():
#     company_name = row["Adherent Name"]
#     company_url = row["URL"]
    
#     try:
#         # Make a GET request to fetch the company's page
#         response = requests.get(company_url, headers=headers)
        
#         # Continue to the next iteration if request fails
#         if response.status_code != 200:
#             print(f"Failed to retrieve data for {company_name}")
#             failed_urls.append(company_url)
#             failed_names.append(company_name)
#             continue

#         # Parse the page content
#         soup = BeautifulSoup(response.text, "html.parser")

#         # Find all contact buttons with mailto: links
#         contact_buttons = soup.find_all("a", href=re.compile(r"^mailto:"))

#         # Extract email addresses from mailto: links
#         for button in contact_buttons:
#             email = button.get("href").replace("mailto:", "")
#             # Append email, name, and URL if email is new
#             if email not in emails:
#                 emails.append(email)
#                 names.append(company_name)
#                 urls.append(company_url)

#     except Exception as e:
#         print(f"An error occurred for {company_name}: {e}")
#         failed_urls.append(company_url)
    
# # Create a DataFrame to store the results
# result_df = pd.DataFrame({
#     "Adherent Name": names,
#     "Email": emails,
#     "URL": urls
# })

# # Save the results to a CSV file
# result_df.to_csv("adherent_names_complete.csv", index=False)

# # Display the first 30 rows
# result_df.head(50)


NameError: name 'adf' is not defined

In [21]:
# Lists to store extracted data
emails = []
names = []
urls = []
no_email_names = []  # Add a list to track names with no email but successfully accessed
failed_urls = []
failed_names = []

for index, row in df.iterrows():
    company_name = row["Adherent Name"]
    company_url = row["URL"]

    try:
        response = requests.get(company_url, headers=headers)
        if response.status_code != 200:
            failed_urls.append(company_url)
            failed_names.append(company_name)
            continue

        soup = BeautifulSoup(response.text, "html.parser")
        contact_buttons = soup.find_all("a", href=re.compile(r"^mailto:"))

        if contact_buttons:  # If emails found
            for button in contact_buttons:
                email = button.get("href").replace("mailto:", "")
                if email not in emails:
                    emails.append(email)
                    names.append(company_name)
                    urls.append(company_url)
        else:  # No email found but URL was accessed
            no_email_names.append(company_name)

    except Exception as e:
        failed_urls.append(company_url)

# Log the number of companies per list
print(f"Total with emails: {len(emails)}")
print(f"Total without emails: {len(no_email_names)}")
print(f"Total failed URLs: {len(failed_urls)}")


Total with emails: 100
Total without emails: 1
Total failed URLs: 12


In [35]:
print(len(result_df))

1856


In [36]:
print(len(failed_names))

196


In [37]:
print(failed_names[0:5])

['573° CONSULTING', '9ALTITUDES', 'A-SYSTEMS', 'ABVSM - COGNILEARNING', 'ACATUS INFORMATIQUE']


In [38]:
failedframe = pd.DataFrame(
    {
        "Names": failed_names,
        "URL":failed_urls
    }
)
failedframe.to_csv("Failed.csv", index=False)
failedframe.head(30)

,Names,URL
0,573° CONSULTING,https://numeum.fr/societe/573-consulting
1,9ALTITUDES,https://numeum.fr/societe/9altitudes
2,A-SYSTEMS,https://numeum.fr/societe/a-systems
3,ABVSM - COGNILEARNING,https://numeum.fr/societe/abvsm---cognilearning
4,ACATUS INFORMATIQUE,https://numeum.fr/societe/acatus-informatique
5,ACII,https://numeum.fr/societe/acii
6,ADDACTIS FRANCE,https://numeum.fr/societe/addactis-france
7,AF2M,https://numeum.fr/societe/af2m
8,AGILITECH,https://numeum.fr/societe/agilitech
9,AKKODIS,https://numeum.fr/societe/akkodis
